<H1> Lab exercise 3: Eigenvalues and SVD</h1>

## Part 3: Computation of eigenvalues

_In the previous part of the lab, we learned how to calculate the eigenvalues and eigenvectors of a matrix in Python. But what methods are used under the hood? In this part we look at two numerical methods used for eigenvalue computations. From mathematics, you know that eigenvalues can be computed via the characteristic equation  𝑑𝑒𝑡(𝐴−𝜆𝐼)=0 . This is not an option in software though, since the coefficients of the characteristic equation can't be computed from determinant evaluations in a numerically stable way. It leads to large perturbations in the roots. In this lab, we will have a look at some computational methods to calculate the eigenvalues of a matrix._ 

<h3>The Power method</h3>

The simplest numerical method for eigenvalue computations is the Power Method. We outline the algorithm below:<br>
<br>
Given a matrix $A$,
- Start with an initial eigenvector $v_{(0)}$, which is a guess, input by the user
- Calculate $v_{(1)}=Av_{(0)}, v_{(2)} = Av_{(1)}, v_{(3)} = Av_{(2)}, \ldots$, and so on. The sequence of $v_i$'s are normalized throughout the process

Using this calculation process, it can be shown that $v_{(i)}$ approaches one eigenvector of the matrix $A$. We terminate the process when we have reached a vector $v_{(i)}$ that is close enough to the true eigenvector (when the difference between two consecutive $v_{(i)}$ is smaller than a given tolerance). When the eigenvector is found, the eigenvalue can be computed with the formula $\lambda=\frac{v^TAv}{v^Tv}$ (the denominator disappears when $v$ is normalized).

You will now implement the Power Method yourself following the instructions below:


_1.1) First, define the matrix_ 
$A = \left( \begin{array}{ccc} 
1 & 2 & 0 \\
1 & 1 & 2 \\
1 & 3 & 1
\end{array} \right) $
_and check eigenvalues/eigenvectors using numpy._

In [2]:
import numpy as np
A = np.array([[1, 2, 0],[1, 1, 2], [1, 3, 1]])
S, V = np.linalg.eig(A)

print("Eigenvalues of matrix A: ", S)
print("Eigenvectors of matrix A: ", V)

Eigenvalues of matrix A:  [ 4.05137424  0.48269595 -1.5340702 ]
Eigenvectors of matrix A:  [[-0.38725131 -0.89283019  0.51116045]
 [-0.59082433  0.23093234 -0.64765823]
 [-0.70778742  0.38668398  0.56502549]]


_1.2) Now implement the Power Method (with a for-loop so you can choose number of iterations). You can find a pseudocode for the implementation below. Make sure you understand how the pseudocode corresponds to the algorithm presented above, and then use it to write an actual Python implementation._
```
  v0 = ...  # initial eigenvector (initial guess), you can use v0=(1, 1,...,1)
  Normalize v0
  N = ...   # Number of iterations
  for k in range(N):
     v = Av0
     v = v/norm(v)  # Normalize v
     e = v.T(Av)    # Compute eigenvalue
     v0 = v
``` 
_1.3) When your code works, run it for different number of iterations ${\tt N}$. You can for example start with 5 iterations and then increase until you have for example 6 correct decimal places (compared with the true eigenvalues calculated above)._

In [1]:
import numpy as np

def power_method(A, v0, N):
    # Normalize the initial guess vector
    v0 = v0 / np.linalg.norm(v0)

    for k in range(N):
        # Calculate Av
        v = np.dot(A, v0)

        # Normalize v
        v = v / np.linalg.norm(v)

        # Compute the eigenvalue
        eigenvalue = np.dot(np.dot(v0, A),v)

        # Update the initial guess vector
        v0 = v

    return eigenvalue, v

# Define your matrix A and initial guess vector v0
A = np.array([[1, 2, 0],[1, 1, 2], [1, 3, 1]])
v0 = np.array([1, 1, 1])

# Number of iterations (you can adjust this value)
N = 5

# Run the Power Method
eigenvalue, eigenvector = power_method(A, v0, N)

print("Estimated Eigenvalue:", eigenvalue)
print("Estimated Eigenvector:", eigenvector) #normalized

Estimated Eigenvalue: 4.050768136828551
Estimated Eigenvector: [0.38747191 0.59043339 0.70799289]


From the results, answer the questions
- Which eigenvalue (and corresponding eigenvector) does the Power Method find? The Power Method is used to find a dominant eigenvalue (one with the largest absolute value).
- Is it exactly the same eigenvector compared with `numpy.linalg.eig`? This one is normalized.

<br>
The Power method is an example of an <i>iterative method</i>, meaning that we get a sequence of solutions $\{\lambda_0, \lambda_1, \ldots \}$ (one solution each iteration), and that the sequence approaches (converges to) the true solution.
<hr>

<h3>The QR-iteration method</h3>


An obvious problem with the Power Method is that we can only find one eigenvalue and one eigenvector at a time. The most common algorithm for computing eigenvalues and eigenvectors is the <i>QR-method</i> outlined below:
- Start with finding the QR-decomposition: $A=QR$
- Flip the order around and calculate a new matrix: $A_{(1)}= RQ$
- Find the QR-decomposition of the new matrix: $A_{(1)}=Q_{(1)}R_{(1)}$
- Flip the order again: $A_{(2)}=R_{(1)}Q_{(1)}$
- And so forth until result is good enough

Almost magically, the eigenvalues can be found on the main diagonal in the matrix $A_{(k)}$ obtained at the $k$th iteration. As with the Power Method, this is also an iterative method which converges to the true solution after a sufficient number of iterations.

_2.1) Implement the QR-method using a for-loop (based on the outline above). Then, run the algorithm for different number of iterations $N$. Compare the diagonal entries of the resulting matrix $A_{(N)}$ with the eigenvalues you computed in Task 1.1). Does the QR-method find all eigenvalues of $A$?_ 

Note that the matrix approaches an upper triangular matrix (the elements under the main diagonal gets smaller as you increase the number of iterations). Remember that the eigenvalues of a triangular matrix can be found on the main diagonal.

In [2]:
def qr_method(A, N):
    for k in range(N):
        Q, R = np.linalg.qr(A)
        A = np.dot(R, Q)
    
    eigenvalues = np.diag(A)
    return eigenvalues, A

# Define your matrix A
A = np.array([[1, 2, 0],[1, 1, 2], [1, 3, 1]])

# Number of iterations (you can adjust this value)
N = 1000

# Run the QR-method
eigenvalues, AN = qr_method(A, N)

print("Estimated Eigenvalues:", eigenvalues)
print(AN)

Estimated Eigenvalues: [ 4.05137424 -1.5340702   0.48269595]
[[ 4.05137424  1.23090532 -0.80888456]
 [ 0.         -1.5340702   0.91136045]
 [ 0.          0.          0.48269595]]


_2.2) Below is the the QR-iteration defined as a function (starting from the input matrix A, and running until convergence). Call the function with_
- The same matrix $A$ as before
- A symmetric matrix, for example $B=A^TA$. Compare the result with the Spectral Theorem, i.e. the resulting matrix should be roughly 
$$
\Lambda= \left( \begin{array}{ccc} 
\lambda_1 &  & 0 \\
          & \ddots & \\
        0  &        & \lambda_n
          \end{array} \right)
          $$
 
**Caution:** The iterative process involves estimating the error at each iteration by comparing the diagonal elements of $A_k$ (approximated eigenvalues) with those of $A_{k-1}$. The iteration terminates if the maximum error falls below a specified tolerance (`tol`). It is crucial to note that this criterion is applicable only when all eigenvalues of $A$ are real (upper triangular Schur form), such as in the case of symmetric matrices. For complex eigenvalues this criterion may not provide reliable convergence assessment.

In [3]:
def QRmethod(A0, tol=1.0e-8):
    # Eigenvalue computation, QR-method
    # Default error tolerance in max-norm is 1.0e-8 

    if tol<1.0e-15:   # Tol cant be smaller than machine epilon
        tol = 1.0e-14
    
    err = 1.0
    while err > tol:
        Q1, R1 = np.linalg.qr(A0)
        A1 = R1@Q1
        A1diag = np.diag(A1)   # Get A1s main diagonal 
        A0diag = np.diag(A0)   # Get A0s main diagonal
        err = (np.linalg.norm(A1diag-A0diag,np.inf))/np.linalg.norm(A1diag,np.inf) # rel. error in max-norm
        A0 = A1
    
    return A1

In [4]:
A = np.array([[1, 2, 0],[1, 1, 2], [1, 3, 1]])
ATA = np.transpose(A)@A
D = QRmethod(ATA, 1.0e-14)
D

array([[ 1.90295050e+01,  1.29365885e-07, -1.44369783e-15],
       [ 1.29365885e-07,  2.80168607e+00,  9.71470822e-11],
       [-1.04877137e-17,  9.71475088e-11,  1.68808981e-01]])

**Some notes on the QR-method we have implemented today:**

- The QR-iteration method above only computes the eigenvalues, but it is possible to also the get the eigenvectors (we will not talk about this today, however). <br>
- The QR-iteration method is very expensive due the the QR-decomposition in each iteration. In the standard method the matrix can be transformed to a **Hessenberg** form before the loop, and that reduces the cost significantly. Also the convergence rate can be improved (leading to fewer iteration). This makes the practical QR-method efficient.<br>

<hr>